In [1]:
import pandas as pd
import os
from functools import reduce

from nlinec import get_models_dir, get_results_dir

In [2]:
ACCURACY_FILE = 'test_accuracy.csv'
ACCURACY_FILE_HYPOTHESIS_ONLY = 'test_accuracy_ho.csv'
MNLI_ACCURACY_FILE = 'mnli_accuracy.csv'

In [3]:
models = sorted(os.listdir(get_results_dir()))
print(models)

['nlinec-AE-2', 'nlinec-B-2', 'nlinec-C-2', 'nlinec-D-1', 'nlinec-D-2', 'nlinec-D-3', 'nlinec-E-2', 'roberta-large-mnli']


In [4]:
accuracy = {}
accuracy_ho = {}
mnli_accuracy = {}
for model in models:
    if os.path.exists(os.path.join(get_results_dir(), model, ACCURACY_FILE)):
        accuracy[model] = pd.read_csv(os.path.join(get_results_dir(), model, ACCURACY_FILE), index_col=0).rename(columns={'accuracy': model})

    if os.path.exists(os.path.join(get_results_dir(), model, ACCURACY_FILE_HYPOTHESIS_ONLY)):
        accuracy_ho[model] = pd.read_csv(os.path.join(get_results_dir(), model, ACCURACY_FILE_HYPOTHESIS_ONLY), index_col=0).rename(columns={'accuracy': model})

    if os.path.exists(os.path.join(get_results_dir(), model, MNLI_ACCURACY_FILE)):
        mnli_accuracy[model] = pd.read_csv(os.path.join(get_results_dir(), model, MNLI_ACCURACY_FILE), index_col=0).rename(columns={'accuracy': model})

In [5]:
# Merge all accuracy dataframes on the 'granularity' column
accuracy_df = reduce(lambda left, right: pd.merge(left, right, on='granularity'), accuracy.values()).T.rename(columns={0: 'Coarse', 1: 'Medium', 2: 'Fine'})
accuracy_ho_df = reduce(lambda left, right: pd.merge(left, right, on='granularity'), accuracy_ho.values()).T.rename(columns={0: 'Coarse HO', 1: 'Medium HO', 2: 'Fine HO'})
mnli_accuracy_df = pd.concat(mnli_accuracy.values(), axis=1).T

In [6]:
# Combine the accuracy and mnli_accuracy dataframes
# Rename the Granularity columns from 0, 1, 2 to 'Coarse', 'Medium', 'Fine'
# Rename the 'accuracy' column from the mnli_accuracy dataframe to 'MNLI Accuracy'
# Remove the 'granularity' row from the resulting dataframe
combined_df = pd.concat([accuracy_df, mnli_accuracy_df], axis=1).rename({'accuracy': 'MNLI'}).drop('granularity', axis=0)
combined_df.round(2)

,Coarse,Medium,Fine,accuracy
nlinec-C-2,0.67,0.69,0.70,0.74
nlinec-D-1,0.88,0.47,0.63,0.84
nlinec-D-2,0.61,0.70,0.71,0.82
nlinec-D-3,0.53,0.63,0.73,0.81
nlinec-E-2,0.16,0.07,0.12,0.70
roberta-large-mnli,0.46,0.35,0.44,0.90


In [7]:
accuracy_ho_df.drop('granularity', axis=0).round(2)

,Coarse HO,Medium HO,Fine HO
nlinec-C-2,0.53,0.59,0.67
nlinec-D-1,0.84,0.36,0.54
nlinec-D-2,0.42,0.59,0.60
nlinec-D-3,0.38,0.50,0.64
nlinec-E-2,0.24,0.05,0.14
roberta-large-mnli,0.22,0.19,0.36
